### CNN Cifar10 두번째 예시

- 기존 예시에서 70% 정확도밖에 보여주지 못했기에 데이터 증폭과 깊어진 레이어로 더 정확성을 높혀보자

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
# 데이터 불러오기

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)



(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 1) (10000, 1)


In [5]:
# 데이터 증폭

gen = ImageDataGenerator(rotation_range = 20,
                        shear_range = 0.2, 
                        width_shift_range = 0.2, 
                        height_shift_range = 0.2,
                        horizontal_flip = True)

augment_ratio = 1.5 # 전체 데이터의 150%
augment_size = int(augment_ratio * x_train.shape[0])

randidx = np.random.randint(x_train.shape[0], size = augment_size)

x_augmented = x_train[randidx].copy()
y_augmented = y_train[randidx].copy()

x_augmented, y_augmented = gen.flow(x_augmented, y_augmented, batch_size = augment_size, shuffle = False).next()


x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))


# 데이터 섞기 (원본 + 새로 생성)
s = np.arange(x_train.shape[0])
np.random.shuffle(s)

x_train = x_train[s]
y_train = y_train[s]


In [6]:
# 모델 생성

cnn = Sequential()

cnn.add(Conv2D(32, (3, 3), activation = "relu", padding = "same", input_shape = (32, 32, 3)))
cnn.add(Conv2D(32, (3, 3), activation = "relu", padding = "same"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25))


cnn.add(Conv2D(64, (3, 3), activation = "relu", padding = "same"))
cnn.add(Conv2D(64, (3, 3), activation = "relu", padding = "same"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, (3, 3), activation = "relu", padding = "same"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, (3, 3), activation = "relu", padding = "same"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256, (3, 3), activation = "relu", padding = "same"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())

cnn.add(Dense(128, activation = "relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation = "softmax"))


In [ ]:
# 컴파일

cnn.compile(loss = "sparse_categorical_crossentropy", optimizer = Adam(), metrics = ["accuracy"])

hist = cnn.fit(x_train, y_train, batch_size = 256, epochs =200, validation_data = (x_test, y_test))

# 실행이 너무 오래 걸려서 실행 보류

Epoch 1/200
1221/1221 [==============================] - 691s 566ms/step - loss: 1.5836 - accuracy: 0.4191 - val_loss: 1.0778 - val_accuracy: 0.6021
Epoch 2/200
 163/1221 [===>..........................] - ETA: 9:54 - loss: 1.2816 - accuracy: 0.5411

In [ ]:
# 모델 평가


import matplotlib.pyplot as plt

plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy Trend')
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend(['train', 'validation'], loc = "best")
plt.grid()
plt.show()

In [ ]:
##

plt.plot(hist.history["loss"])
plt.plot(hist.history['val_loss'])
plt.title('Loss Trend')
plt.ylabel("loss")
plt.xlabel("epochs")
plt.legend(['train', 'validation'], loc = "best")
plt.grid()
plt.show()